Carichiamo i dati e iniziamo a fare qualche carotaggio per verificare quali informazioni sono presenti

In [1]:
import pandas as pd

In [25]:
df_trasformati = pd.read_csv('Rapporti_di_lavoro_trasformati.csv')

In [17]:
df_trasformati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975342 entries, 0 to 1975341
Data columns (total 9 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   DATA                       object
 1   GENERE                     object
 2   ETA                        int64 
 3   SETTOREECONOMICODETTAGLIO  object
 4   TITOLOSTUDIO               object
 5   CONTRATTO                  object
 6   MODALITALAVORO             object
 7   PROVINCIAIMPRESA           object
 8   ITALIANO                   object
dtypes: int64(1), object(8)
memory usage: 135.6+ MB


In [32]:
df_trasformati.d

,ETA
count,1.975342e+06
mean,3.592761e+01
std,1.191772e+01
min,0.000000e+00
25%,2.600000e+01
50%,3.400000e+01
75%,4.400000e+01
max,1.000000e+02


In [29]:
df_trasformati.groupby(['SETTOREECONOMICODETTAGLIO']).size()


SETTOREECONOMICODETTAGLIO
Acquacoltura in acqua di mare, salmastra o lagunare e servizi connessi                                 43
Acquacoltura in acque dolci e servizi connessi                                                         60
Affari esteri                                                                                          61
Affittacamere per brevi soggiorni, case ed appartamenti per vacanze, bed and breakfast, residence    1542
Affitto di aziende                                                                                    102
                                                                                                     ... 
Trattamento igienico del latte                                                                        306
Trivellazioni e perforazioni                                                                          871
Università popolare                                                                                     3
Utilizzo di aree for

In [33]:
ateco = pd.read_excel('CodiceAteco.xlsx')

In [34]:
ateco

,Lettera,Codice Ateco 2007,Descrizione
0,A,A,"AGRICOLTURA, SILVICOLTURA E PESCA"
1,A,01,COLTIVAZIONI AGRICOLE E PRODUZIONE DI PRODOTTI...
2,A,01.1,COLTIVAZIONE DI COLTURE AGRICOLE NON PERMANENTI
3,A,01.11,"Coltivazione di cereali (escluso il riso), leg..."
4,A,01.11.1,Coltivazione di cereali (escluso il riso)
...,...,...,...
1885,T,97.00.02,Attività di condomini
1886,T,98,PRODUZIONE DI BENI E SERVIZI INDIFFERENZIATI P...
1887,T,98.1,PRODUZIONE DI BENI INDIFFERENZIATI PER USO PRO...
1888,T,98.2,PRODUZIONE DI SERVIZI INDIFFERENZIATI PER USO ...


In [35]:
df_trasformati_ateco = df_trasformati.merge(ateco, left_on='SETTOREECONOMICODETTAGLIO', right_on='Descrizione', how='left')

In [36]:
df_trasformati_ateco

,DATA,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,Lettera,Codice Ateco 2007,Descrizione
0,10/04/2017,M,41,Realizzazione di coperture,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,MAROCCO,F,43.91,Realizzazione di coperture
1,10/01/2018,M,16,Ristorazione con somministrazione,LICENZA MEDIA,LAVORO INTERMITTENTE,NON DEFINITO,BERGAMO,ITALIA,I,56.10.11,Ristorazione con somministrazione
2,07/01/2020,F,18,Rifugi di montagna,LICENZA MEDIA,LAVORO INTERMITTENTE,NON DEFINITO,BERGAMO,ITALIA,I,55.20.3,Rifugi di montagna
3,03/01/2010,F,60,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,NaN,NaN,NaN
4,07/03/2014,M,60,Altre attività di lavori specializzati di cost...,LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,LECCO,ITALIA,F,43.99.09,Altre attività di lavori specializzati di cost...
...,...,...,...,...,...,...,...,...,...,...,...,...
1975337,09/08/2017,M,22,Commercio al dettaglio di carburante per autot...,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,PAKISTAN,G,47.30.0,Commercio al dettaglio di carburante per autot...
1975338,01/10/2019,F,42,Produzione di software non connesso all'edizione,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,TIROCINIO,TEMPO PIENO,MONZA E BRIANZA,ITALIA,J,62.01,Produzione di software non connesso all'edizione
1975339,02/06/2012,M,40,Lavori di meccanica generale,LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MONZA E BRIANZA,ITALIA,C,25.62,Lavori di meccanica generale
1975340,02/05/2021,M,21,Fabbricazione di altre pompe e compressori,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MONZA E BRIANZA,PAKISTAN,C,28.13,Fabbricazione di altre pompe e compressori


In [37]:
df_trasformati_ateco.groupby(['Lettera']).size()

Lettera
A     25386
B       604
C    209156
D      1500
E      8911
F     93358
G    199844
H    163211
I    203070
J     54441
K     12957
L      4457
M     55672
N    221883
O     24242
P    224668
Q     66248
R     32922
S     52942
dtype: int64

In [39]:
pd.crosstab(df_trasformati_ateco.Lettera, df_trasformati_ateco.GENERE)

GENERE,F,M
Lettera,,
A,5220,20166
B,88,516
C,60710,148446
D,615,885
E,1186,7725
F,5059,88299
G,118009,81835
H,38280,124931
I,93272,109798


In [41]:
df_trasformati_ateco.DATA.max()

'12/12/2020'

In [42]:
df_trasformati_ateco.DATA.min()

'01/01/2010'

In [ ]:
df_trasformati_ateco['ANNO'] = df_trasformati

In [45]:
pd.crosstab(df_trasformati_ateco.TITOLOSTUDIO, df_trasformati_ateco.GENERE, margins=True)

GENERE,F,M,All
TITOLOSTUDIO,,,
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CHE PERMETTE L'ACCESSO ALL'UNIVERSITA,302834,213558,516392
DIPLOMA DI SPECIALIZZAZIONE,5501,4111,9612
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO,655,401,1056
DIPLOMA UNIVERSITARIO,9142,5174,14316
LAUREA - Vecchio o nuovo ordinamento,149645,69040,218685
LICENZA ELEMENTARE,3729,14309,18038
LICENZA MEDIA,194024,340259,534283
MASTER UNIVERSITARIO DI PRIMO LIVELLO,1246,653,1899
NESSUN TITOLO DI STUDIO,161994,322216,484210
